In [1]:
type Point{T<:Real}
    x::T
    y::T
    z::T
end

struct Cluster
    points::Array{Point, 1}
    center::Point
end    

In [2]:
#=
    Function that returns array of arrays
    of string values by reading of .csv file
=#
function getDataSetFromCSV(filename::String)::Array{Array{String, 1}, 1}
    arrayOfData::Array{Array{String, 1}, 1} = []
    open(filename) do file   
        for line::String in eachline(file)
          push!(arrayOfData, split(line, ','))  
        end    
    end   
    arrayOfData
end    

#=
    Function that converts array of arrays
    of string values to the 2-dimensional array
    of parsed values to float format 
=#
function convertToFloatData(array::Array{Array{String, 1}, 1},
     rowNumber::Int64, colNumber::Int64)::Array{Float64, 2}
    arrayOfFloats = Array{Float64, 2}(rowNumber, colNumber);   
    for i::Int64 = 1 : rowNumber
        for j::Int64 = 1 : colNumber
            arrayOfFloats[i, j] = parse(Float64, array[i][j]) 
        end
    end    
    arrayOfFloats
end

#=
    Function that gets index of the column from dataset
    by its name (if it contains it)    
=#
function getColumnIndexByName(array::Array{Array{String, 1}, 1},
     colName::String)::Int64 
    for i::Int64 = 1 : size(array[1], 1)
        if(array[1][i] == colName)           
            return i
        end
    end   
    return -1
end

#=
    Function that returns column's data by its index in dataset    
=#
function getDataFromColumnByIndex(array::Array{Array{String, 1}, 1},
     index::Int64)::Array{Float64, 1} 
    dataFromColumn = Array{Float64, 1}(0);
    for i::Int64 = 2 : size(array, 1)
        push!(dataFromColumn, parse(Float64, array[i][index])) 
    end   
    dataFromColumn
end

#=
    Function that returns column's data by its name in dataset    
=#
function getDataFromColumnByName(array::Array{Array{String, 1}, 1},
     colName::String)::Array{Float64, 1}       
    dataFromColumn = Array{Float64, 1}(0);
    colIndex::Int64 = getColumnIndexByName(array, colName) 
    for i::Int64 = 2 : size(array, 1)          
        push!(dataFromColumn, parse(Float64, array[i][colIndex])) 
    end   
    dataFromColumn
end

#=
    Function that links columns of the dataset into 2-d matrix
=#
function linkData(arrays...)::Matrix{Float64}
    result = Matrix{Float64}(size(arrays[1], 1), length(arrays))
    for i::Int64 = 1 : size(arrays[1], 1)
        for j::Int64 = 1 : length(arrays)
            result[i, j] = arrays[j][i]
        end
    end    
    result
end

#=
    Function that creates array of the points based on the chosen dataset,
    represented in 2-d matrix
=#
function determinePoints(dataset::Matrix{Float64})::Array{Point, 1}
    len::Int64 = size(dataset, 1)
    points = Array{Point, 1}(len)  
    for i::Int64 = 1 : len
        point = Point(dataset[i, 1], dataset[i, 2], dataset[i, 3])      
        points[i] = point      
    end  
    points
end

#=
    Function that creates array of points' projections
    based on the selected axis ("x", "y", "z")
=#
function getPointsProjections(points::Array{Point, 1},
     axis::String)::Array{Float64, 1}
    coords = Array{Float64, 1}(0)
    for p::Point in points
        if(lowercase(axis) == "x")
            push!(coords, p.x)
        elseif(lowercase(axis) == "y")
            push!(coords, p.y)
        elseif(lowercase(axis) == "z")
            push!(coords, p.z)
        else    
            push!(coords, 0.0)      
        end
    end
    coords
end

getPointsProjections (generic function with 1 method)

In [3]:
# Reading dataset from .csv file
fileData = getDataSetFromCSV("creditcard.csv")


xs = getDataFromColumnByName(fileData, "\"V1\"")
ys = getDataFromColumnByName(fileData, "\"V3\"")
zs = getDataFromColumnByName(fileData, "\"V6\"")

print(linkData(xs, ys, zs))

dataset = linkData(xs, ys, zs)
points = determinePoints(dataset)
println(getPointsProjections(points, "x"))


# Removing headers from dataset
# shift!(fileData)
# Converting string values to floats of the dataset
# dataset = convertToFloatData(fileData, 200, 10)
# println(dataset)

[-1.35981 2.53635 0.462388; 1.19186 0.16648 -0.0823608; -1.35835 1.77321 1.8005; -0.966272 1.79299 1.2472; -1.15823 1.54872 0.0959215; -0.425966 1.14111 -0.0297276; 1.22966 0.0453708 0.272708; -0.644269 1.07438 0.428118; -0.894286 -0.113192 3.72182; -0.338262 1.04437 -0.246761; 1.44904 0.91386 -0.629152; 0.384978 -0.8743 3.31703; 1.25 0.38393 -0.75323; 1.06937 0.828613 0.337544; -2.79185 1.64175 0.807596; -0.752417 2.05732 -0.0778498; 1.10322 1.26733 0.288069; -0.436905 0.924591 -0.127867; -5.40126 1.1863 -1.76341; 1.49294 0.454795 -0.720961; 0.694885 1.02922 1.30911; 0.962496 -0.171479 1.69604; 1.16662 -0.0673003 0.0894735; 0.247491 1.18547 -0.150116; -1.94653 -0.40557 2.95505; -2.07429 1.32202 -0.959537; 1.17328 0.283905 -0.916054; 1.32271 0.434555 -0.831083; -0.414289 1.72745 -0.200331; 1.05939 1.26613 0.578435; 1.23743 0.380526 -0.494084; 1.11401 0.493702 -0.0107538; -0.529912 1.34725 0.100223; -0.529912 1.34725 0.100223; -0.535388 1.35108 0.0869829; -0.535388 1.35108 0.0869829; -0